# Exploring Lake Detection in Radar Sounding Data from East Antarctica   

In this notebook, we are going lake hunting! Previous work in East Antarctica near the Dome C region found evidence for a subglacial lake that researchers named Horseshoe Lake. You will look at some airborne ice-penetrating radar data collected by the UTIG High Capability Radar Sounder (HiCARS) in the austral spring of 2009 and see if you can figure out where Horseshoe Lake is located.

## Part I: Setting Up the Environment  

As usual, we need to mount our Google Drive and import the various libraries we want to use. Make sure that you have saved the data file (HoreshoeLake.mat) to the Google drive associated with the account you use for Google Colab in a directory called /EAS4940/SubglacialLakeDemo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install hdf5storage

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import hdf5storage as hd
from google.colab import output

The code below will load up the radar data. You have access to a number of variables including:  

data['Data'] - a 2D matrix of the recorded radar echo power, where the first dimension is time (e.g. distance from the radar or depth in the ice) and the second dimension is the distance along the flight line.  

data['Distance'] - a 1D array that records the distance along the flight line in meters for each column in data['Data'].   

data['Time'] - a 1D array that records the two-way travel time from the radar from every row in data['Data'].   

data['Surface'] - the index (row) of the surface echo for every column in data['Data'].   

data['Bottom'] - the index (row) of the bed echo for every column in data['Data'].   


In [ ]:
data = hd.loadmat('/content/drive/MyDrive/EAS4940/SubglacialLakeDemo/HorseshoeLake.mat');
data['Surface'] = np.matrix.squeeze(data['Surface'])
data['Bottom'] = np.matrix.squeeze(data['Bottom'])
data['Time'] = np.matrix.squeeze(data['Time'])

## Part II: Visualizing the Data and Calculating Ice Thickness   

The first code block below will plot up the radar data as an image. Just from looking at the image, can you guess where Horseshoe Lake might be located? Write down your best guess and why.

In [ ]:
c = 3e8    # speed of light in a vacuum
n = 1.78   # index of refraction in ice

fig, ax = plt.subplots(layout="constrained")
im = ax.imshow(data['Data'], cmap='Greys', vmin=110, vmax=200)
ax.set_title('Radar Data over Horshoe Lake')
ax.set_xlabel("Trace Number")
ax.set_ylabel("Fast Time Sample")
cbar = fig.colorbar(im, location='bottom')
cbar.set_label("Power (dB)")

# Set the aspect ratio to something normal for radargrams before plotting, otherwise it will be super tall and skinny, which is hard to read
plt.gca().set_aspect(0.6);
plt.xticks(rotation=45)
plt.ylim([2500,0])
plt.show()

The first thing we will do is calculate the ice thickness at every point along the flight line. Below is some starter code. Fill in the line to calculate ice thickness. If we know the speed of light in a vacuum (c) and refractive index of ice (n), we can calculate thickness (T) from the two-way travel time to the bed (B) and surface (S) as follows:  

$T = 0.5\frac{c}{n}(B - S)$  

We might expect lakes to be located in local topographic lows, e.g. under some of the thickest ice. Is the ice thickness consistent with your guess for the location of Horseshoe Lake?

In [ ]:
# Calculate Thickness
c = 299792458     # speed of light in vacuum, m/s
n = 1.78          # refractive index of ice

# Caculate the two=-way travel time (TWTT) to the surface and bed
twtt_surface = np.zeros(data['Surface'].size)         # two-way travel time from radar to surface and back
twtt_bed = np.zeros(data['Surface'].size)             # two=way travel time from radar to bed and back
for i in range(0,twtt_surface.size):
  twtt_surface[i] = data['Time'][data['Surface'][i]]
  twtt_bed[i] = data['Time'][data['Bottom'][i]]

############## Fill code to calculate ice thickness here! #################


# Plot the resuts
plt.close('all')
fig, ax1 = plt.subplots(layout='constrained')
ax1.plot(data['Distance']/1000, thickness);
ax1.set_title('Ice Thickness')
ax1.set_ylabel('Ice Thickness (m)')
ax1.set_xlabel('Distance Along-Track (km)')

fig.set_size_inches(10,4, forward=True)
plt.xticks(rotation=45)
plt.show()

## Part 3: Analyzing Bed Power   

Since water has a much higher dielectric constant than rock, we would expect more power to be reflected back to the radar from a lake than from surrounding bedrock. By looking at the change in the power of the bed echo along the flight line, we can hopefully confirm our guess for the lake. In the code below, add a line to extract the bed echo power at every column of your data['Data] matrix. Then we will call a function to apply a geometric correction that accounts for differences in bed power that might occur just because the one part of the bed was further away from the radar than another. Looking at your corrected bed echo power, do you see any regions that have significantly higher power than their surroundings? Are those areas consistent with your guess for Horseshoe Lake?

In [ ]:
def GeometricCorrection(surface, bed, power, time):
  c = 3e8
  n = 1.78
  for i in range(0, power.size):
    h = 0.5*c*time[surface[i]]
    z = 0.5*(c/n)*(time[bed[i]] - time[surface[i]])
    correction = (2*(h + z/n))**2
    return power + 10*np.log10(correction)


# Get the bed power
bed_power = np.zeros(data['Bottom'].shape)
for i in range(0,bed_power.size):
  ##########################################################################################
  # Write in a line of code here to extract the power of the bed echo in each column of data
  # Hint: use the bed index array to index into your data matrix


bp_corr = GeometricCorrection(data['Surface'], data['Bottom'], bed_power, data['Time'])

# Plot the resuts
plt.close('all')
fig, ax1 = plt.subplots(layout='constrained')
ax1.plot(data['Distance']/1000, bp_corr);
ax1.set_title('Bed Power (No Attenuation Correction)')
ax1.set_ylabel('Power (dB)')
ax1.set_xlabel('Distance Along-Track (km)')

fig.set_size_inches(10,4, forward=True)
plt.xticks(rotation=45)
plt.show()

## Part 4: Correct for Attenuation  

To properly interpret the bed power, we also need to correct for attenuation of the radar waves through the ice column. If the ice is thicker in one location and thinner in another, the wave will attenuate a different amount before reaching the bed. Therefore, we might see variations in bed echo power that are just related to ice thickness, not to bed properties. The code below estimates the attenuation rate in dB/m by fitting a line to the power decay of the internal/englacial reflectors.   

The plot below will show you power as a function of depth from the englacial reflectors, and the best fit line for that decay rate.

In [ ]:
# Estimate the attenuation
c = 3e8
n = 1.78
dt = np.mean(np.diff(data['Time']))

# Find the power of all englacial reflectors starting 100 meters below the surface (~bottom of firn)
# and ending 300 m above the bed to avoid the echo free zone
start = np.rint(data['Surface'][0] + (2*100/c/dt)).astype("int")
stop = np.rint(data['Bottom'][0] - (2*300/c/dt)).astype("int")
peaks,_ = scipy.signal.find_peaks(data['Data'][start:stop,0])
peaks = peaks + start
depth = 0.5*(c/n)*(data['Time'][peaks] - data['Time'][data['Surface'][0]])
power = data['Data'][peaks,0]

for index in range(0, bed_power.size):
  start = data['Surface'][index] + 100
  stop = data['Bottom'][index] - 500
  peaks,_ = scipy.signal.find_peaks(data['Data'][start:stop,index])
  peaks = peaks + start
  depth_add = 0.5*(c/n)*(data['Time'][peaks] - data['Time'][data['Surface'][index]])
  power_add = data['Data'][peaks,index]
  depth = np.concatenate((depth, depth_add))
  power = np.concatenate((power, power_add))

coef = np.polyfit(depth,power,1)     # Find best fit line to englacial reflector power vs. depth
atten_rate = coef[0]                 # Mean attenuation rate is slope of best fit line
poly1d_fn = np.poly1d(coef)

# Plot the resuts
plt.close('all')
fig, ax1 = plt.subplots(layout='constrained')
plt.plot(depth,power,'yo', depth, poly1d_fn(depth), '--k')
ax1.set_title('Attenuation Rate Estimates')
ax1.set_ylabel('Power (dB)')
ax1.set_xlabel('Distance Along-Track (km)')

fig.set_size_inches(5,4, forward=True)
plt.xticks(rotation=45)
plt.show()


In the code block below, add a new line to apply the attenuation correction to the bed. If your attenuation rate is A dB/m and your ice thickness is T, then your corrected power ($P_C$) as a function of the geometrically corrected bed power ($P_G$) is:

$P_C = P_G - AT$    

How does the attenuation corrected bed power compare to the geometrically corrected bed power? Does this improve your confidence in the lake location?

In [ ]:
# Correct for Attenuation

##############################################################
# Finish this code line to correct your data for attenuation!
bp_ac =

# Plot the resuts
plt.close('all')
fig, ax1 = plt.subplots(layout='constrained')
ax1.plot(data['Distance'],bp_ac)
ax1.set_title('Bed Power After Attenuation Correction')
ax1.set_ylabel('Power (dB)')
ax1.set_xlabel('Distance Along-Track (km)')

fig.set_size_inches(10,4, forward=True)
plt.xticks(rotation=45)
plt.show()

## Part 5: Waveform Abruptness  

Another metric we can use for water detection is waveform abruptness. If the reflection is very sharp and peaky, that is more consistent with water. If the reflection is diffuse or has a long tail, that is more consistent with rough surface scattering from bedrock. Run the code below and look at the abruptness. Where are the values highest? How does that compare to where the attenuation-corrected bed power is highest? What is your final guess for the location of Horseshoe Lake?

In [ ]:
# We will integrated our echo starting 15 meters above the waveform peak and finishing 45 m below the waveform peak
left_buf = 15
right_buf = 50

# Normalize by expected max abruptness given this radar system's parameters
P_max = 0.04

abruptness = np.zeros(bp_ac.shape)   # Stores abruptness of lakes surface reflector
# Loop through each trace in each lake:
for k in range(bp_ac.size):
    # Find the fast time sample number of the lake surface reflection
    peak_ind = data['Bottom'][k]
    # Set up the bounds on our region of interest for calculating aggregate power, based on the distance buffers we defined
    y_start = int(peak_ind - np.rint(left_buf/(0.5*(c/n)*np.mean(np.diff(data['Time'],axis=0)))))
    y_stop = int(peak_ind + np.rint(right_buf/(0.5*(c/n)*np.mean(np.diff(data['Time'],axis=0)))))
    # Calculate the aggregate power
    P_agg = sum(data['Data'][y_start:y_stop,k])
    # Calculate the abrtupness
    abruptness[k] = bed_power[k]/P_agg/P_max

# Close our previous figure to save memory
plt.close('all')
fig, ax = plt.subplots(layout="constrained")
ax.plot(data['Distance'], abruptness)
ax.plot(data['Distance'], pd.DataFrame(abruptness).rolling(50,center=True).mean())
ax.set_title('Waveform Abruptness')
ax.set_xlabel("Trace Number")
ax.set_ylabel("Abruptness")
fig.set_size_inches(10,4, forward=True)
plt.xticks(rotation=45)
plt.show()
